In [ ]:
from lets_plot import *
from lets_plot.geo_data import *

import pandas as pd

LetsPlot.setup_html()

In [ ]:
income_all = pd.read_csv('../data/US_household_income_2017.csv', encoding='latin-1')
income_all.head(3)

In [ ]:
income_by_state = income_all.groupby("State_Name", as_index=False)["Mean"].mean()
income_by_state.head(3)

In [ ]:
# load coordinates of US states in low resolution
states = regions_state('US-48').boundaries(resolution=4)
states.head(3)

#### Blank map

In [ ]:
map_theme = theme(axis_line="blank", axis_text="blank", axis_title="blank", axis_ticks="blank") + ggsize(900, 400)
ggplot() + geom_map(map=states) + map_theme

In [ ]:
ggplot(income_by_state) + geom_map(aes(fill="Mean"), map=states, map_join=["State_Name", "request"]) + map_theme \
    + scale_fill_gradient(low="#007BCD", high="#FE0968", name="Mean income")

In [ ]:
scale_fill_gradient2?

In [ ]:
# Issue: 'request' in the result is empty.
counties = regions_county(within="US-48").boundaries(resolution=4)
counties.head(3)

In [ ]:
ggplot() + geom_map(map=counties) + map_theme

In [ ]:
income_by_county = income_all.groupby(["State_Name","County"], as_index=False)["Mean"].mean()
income_by_county.head(3)

In [ ]:
# Issue: 'Internal error', message uninformative
ggplot(income_by_county) + geom_map(aes(fill="Mean"), map=counties, map_join=["County", "request"]) + map_theme \
    + scale_fill_gradient(low="#007BCD", high="#FE0968", name="Mean income", na_value="white")

In [ ]:
# Issue: 'map_join' can't join tables just by county name, without state name. 
ggplot(income_by_county) + geom_map(aes(fill="Mean"), map=counties, map_join=["County", "found name"]) + map_theme \
    + scale_fill_gradient(low="#007BCD", high="#FE0968", name="Mean income", na_value="white")

In [ ]:
# Issue: batch query takes a lot of time and results with an error. The message is misleading:
# ValueError: Error: Service is down for maintenance
#regions_county(income_by_county["County"].tolist(), within=income_by_county["State_Name"].tolist())

In [ ]:
# Geocode USA only once
usa = regions_country('usa')

In [ ]:
def display_progress(i, n):
    def display(s):
        from IPython.display import display, clear_output, HTML
        if s:
            clear_output(wait=True)
            display(HTML("<font color='green'>{}</font>".format(s)))
        else:
            clear_output()
        
    if i != n:
        display('Geocoding progress: {}%'.format(round(i / n * 100, 1)))
    if i == n:
        display(None)

In [ ]:
# Search states once for faster counties geocoding. 
# States are duplicated but geocoding at level 'state' is pretty fast - dedup gives only 2x speed-up.
states = regions_builder('state', income_by_county["State_Name"].tolist(), within=usa)\
    .chunk_request(display_progress) \
    .build()

In [ ]:
# Geocode counties with already geocoded states for better performance.
counties = regions_builder('county', income_by_county["County"].tolist(), within=states)\
    .chunk_request(display_progress) \
    .build()